In [1]:
import pandas as pd
import numpy as np
import datetime
import math
import os
import matplotlib.pyplot as plt
from pylab import rcParams

plt.style.use('ggplot')
%pylab inline --no-import-all

#風力発電初期値
Pr=2.3 #定格電力
V_CIN=2.5 #カットイン風速
V_rat=20 #
V_CO=20 #カットアウト風速
H=30.5 #高さ
H_top=80 #風速計測の高さ（高い）
H_ref=10 #風速計測の高さ（低い）
atm_pre0=1.225 #基準大気圧
total_cost = 1575000 #コスト[yen/unit]
Op_year = 10 #稼動年数
cost = (total_cost/(Op_year*365*24)) #コスト[yen/hour/unit]

Populating the interactive namespace from numpy and matplotlib


In [2]:
pwd()

'/home/yuichiro/Miniconda/Weather/GribToCSV'

# Set or Modify parameters below!!

In [3]:
#インプットデータ
year="2019"
month="04"
day=21
########Check time value
day2=str(day+2)
day3=str(day+3)
day=str(day)
time="120000"
file_dir_path="Output_CSV/"+year+"/"+month+"/"+day
#開始日本時間（UTC+9）
##time=000000 09:00-12:00
#start_TimeJP =year+'-'+month+'-'+day+' 09:00'
#finish_TimeJP =year+'-'+month+'-'+day2+' 12:00'

##time=000000 21:00-00:00
start_TimeJP =year+'-'+month+'-'+day+' 21:00'
finish_TimeJP =year+'-'+month+'-'+day3+' 00:00'

In [4]:
###UTC time to Japan in time:000000
TimeJP = pd.date_range(start_TimeJP,finish_TimeJP,freq='H')
TimeJP.columns = ['Datetime in Japan']
Date =pd.DataFrame({'DateTime': TimeJP.values})
#Date
Date['DateTime'] = pd.to_datetime(Date['DateTime'])
#Date.dtypes
if len(Date)==52:
    print("OK")
else:
    print("Error")
    print(len(Date))

OK


In [5]:
Date['Date']=Date['DateTime']
Date['Year']=Date['DateTime'] .dt.year
Date['Month']=Date['DateTime'] .dt.month
Date['Day']=Date['DateTime'] .dt.day
Date['Hour']=Date['DateTime'] .dt.hour

In [6]:
input_tempFH_0015 = pd.read_csv(file_dir_path+'/FH_00-15_temperature'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_tempFH_1633 = pd.read_csv(file_dir_path+'/FH_16-33_temperature'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_tempFH_3439 = pd.read_csv(file_dir_path+'/FH_34-39_temperature'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_tempFH_4051 = pd.read_csv(file_dir_path+'/FH_40-51_temperature'+year+'_'+month+'_'+day+'_'+time+'.csv')

input_temp = pd.concat([input_tempFH_0015,input_tempFH_1633,input_tempFH_3439,\
                       input_tempFH_4051],axis=1).T.drop(["index","lons","lats"])
input_temp.columns = ['temperature']
input_temp=input_temp.reset_index()
#input_temp

In [7]:
input_UwindFH_0015 = pd.read_csv(file_dir_path+'/FH_00-15_Uwind'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_UwindFH_1633 = pd.read_csv(file_dir_path+'/FH_16-33_Uwind'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_UwindFH_3439 = pd.read_csv(file_dir_path+'/FH_34-39_Uwind'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_UwindFH_4051 = pd.read_csv(file_dir_path+'/FH_40-51_Uwind'+year+'_'+month+'_'+day+'_'+time+'.csv')

input_Uwind = pd.concat([input_UwindFH_0015,input_UwindFH_1633,input_UwindFH_3439,\
                       input_UwindFH_4051],axis=1).T.drop(["index","lons","lats"]).apply(abs)
input_Uwind.columns = ['Uwind']

input_Uwind=input_Uwind.reset_index()

In [8]:
input_VwindFH_0015 = pd.read_csv(file_dir_path+'/FH_00-15_Vwind'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_VwindFH_1633 = pd.read_csv(file_dir_path+'/FH_16-33_Vwind'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_VwindFH_3439 = pd.read_csv(file_dir_path+'/FH_34-39_Vwind'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_VwindFH_4051 = pd.read_csv(file_dir_path+'/FH_40-51_Vwind'+year+'_'+month+'_'+day+'_'+time+'.csv')

input_Vwind = pd.concat([input_VwindFH_0015,input_VwindFH_1633,input_VwindFH_3439,\
                       input_VwindFH_4051],axis=1).T.drop(["index","lons","lats"]).apply(abs)
input_Vwind.columns = ['Vwind']
input_Vwind=input_Vwind.reset_index()

In [9]:
input_radFH_0015 = pd.read_csv(file_dir_path+'/FH_00-15_radiation'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_radFH_1633 = pd.read_csv(file_dir_path+'/FH_16-33_radiation'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_radFH_3439 = pd.read_csv(file_dir_path+'/FH_34-39_radiation'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_radFH_4051 = pd.read_csv(file_dir_path+'/FH_40-51_radiation'+year+'_'+month+'_'+day+'_'+time+'.csv')

input_rad = pd.concat([input_radFH_0015,input_radFH_1633,input_radFH_3439,\
                       input_radFH_4051],axis=1).T.drop(["index","lons","lats"])
input_rad.columns = ['radiation']
input_rad=input_rad.reset_index()

# Wind Calc

In [10]:
##calc wind
wind10 = np.square(input_Uwind['Uwind'].values)+np.square(input_Vwind['Vwind'].values)
wind10 = np.sqrt(wind10)
wind10 =pd.DataFrame({'wind': wind10})
wind10.columns = ['wind_10m[m/s]']
#ind10

In [11]:
forecast=pd.concat([Date,wind10,input_temp,input_rad,input_Uwind,input_Vwind],axis=1,sort=False)

In [12]:
#Using Vz = Vr(Z / Zr)^(1/n) n=4

wind30_values = wind10.values*((H/10)**(1/3))
wind30 = pd.DataFrame( {'wind_30m[m/s]': wind30_values.reshape(-1,) })

In [13]:
Pwt_values=(np.where((V_rat<=wind30_values) & (wind30_values <= V_CO), Pr,\
              np.where((V_CIN<=wind30_values) & (wind30_values <= V_rat), \
                       Pr*(wind30_values-V_CIN)/(V_rat-V_CIN),0 )))
Pwt = pd.DataFrame( {'wind output[kWh]': Pwt_values.reshape(-1,) })

In [14]:
forecast=pd.concat([forecast,wind30,Pwt],axis=1,sort=False)
#forecast=forecast.set_index('DateTime')
forecast

,DateTime,Date,Year,Month,Day,Hour,wind_10m[m/s],index,temperature,index,radiation,index,Uwind,index,Vwind,wind_30m[m/s],wind output[kWh]
0,2019-04-21 21:00:00,2019-04-21 21:00:00,2019,4,21,21,0.536934,temperature FH0,15.915853,radiation FH0-1W/m2(avg),0.000000,Uwind FH0m/s,0.459244,Vwind FH0m/s,0.278196,0.778671,0.000000
1,2019-04-21 22:00:00,2019-04-21 22:00:00,2019,4,21,22,0.726766,temperature FH1,15.322592,radiation FH1-2W/m2(avg),0.000000,Uwind FH1m/s,0.706877,Vwind FH1m/s,0.168861,1.053969,0.000000
2,2019-04-21 23:00:00,2019-04-21 23:00:00,2019,4,21,23,1.124510,temperature FH2,15.305013,radiation FH2-3W/m2(avg),0.000000,Uwind FH2m/s,0.485459,Vwind FH2m/s,1.014324,1.630785,0.000000
3,2019-04-22 00:00:00,2019-04-22 00:00:00,2019,4,22,0,0.665433,temperature FH3,14.687673,radiation FH3-4W/m2(avg),0.000000,Uwind FH3m/s,0.463940,Vwind FH3m/s,0.477034,0.965023,0.000000
4,2019-04-22 01:00:00,2019-04-22 01:00:00,2019,4,22,1,1.172903,temperature FH4,14.866750,radiation FH4-5W/m2(avg),0.000000,Uwind FH4m/s,0.453102,Vwind FH4m/s,1.081850,1.700965,0.000000
5,2019-04-22 02:00:00,2019-04-22 02:00:00,2019,4,22,2,1.056662,temperature FH5,14.455984,radiation FH5-6W/m2(avg),0.000000,Uwind FH5m/s,0.728182,Vwind FH5m/s,0.765694,1.532391,0.000000
6,2019-04-22 03:00:00,2019-04-22 03:00:00,2019,4,22,3,0.792906,temperature FH6,13.871572,radiation FH6-7W/m2(avg),0.000000,Uwind FH6m/s,0.792904,Vwind FH6m/s,0.001605,1.149886,0.000000
7,2019-04-22 04:00:00,2019-04-22 04:00:00,2019,4,22,4,1.209161,temperature FH7,14.091635,radiation FH7-8W/m2(avg),0.000000,Uwind FH7m/s,0.695457,Vwind FH7m/s,0.989145,1.753546,0.000000
8,2019-04-22 05:00:00,2019-04-22 05:00:00,2019,4,22,5,0.937732,temperature FH8,13.733907,radiation FH8-9W/m2(avg),1.437500,Uwind FH8m/s,0.937696,Vwind FH8m/s,0.008304,1.359916,0.000000
9,2019-04-22 06:00:00,2019-04-22 06:00:00,2019,4,22,6,0.974783,temperature FH9,13.694204,radiation FH9-10W/m2(avg),55.643873,Uwind FH9m/s,0.944780,Vwind FH9m/s,0.239984,1.413648,0.000000


# Solar calc

In [15]:
#太陽光発電初期値

#インプットデータ
#input_rad = pd.read_csv('rad.csv')#encordingはUTF-8なので注意 unitは、MJ/m2
input_csv_parameter = pd.read_csv('PV_input_parameter.csv')

In [16]:
#1列目以外を抽出
input_csv_parameter = input_csv_parameter.iloc[:, [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]]

Gsc = float(input_csv_parameter.iat[0,0])#float(1.367)#the solar constant [1.367 kW/m2]
Gt_NOCT = float(input_csv_parameter.iat[0,1]) #the solar radiation at which the NOCT is defined [0.8 kW/m2]
Ta_NOCT = float(input_csv_parameter.iat[0,2])+273.15 #the ambient temperature at which the NOCT is defined [20°C]
Tc_NOCT = float(input_csv_parameter.iat[0,3])+273.15 #the nominal operating cell temperature [°C]
Tc_STC = float(input_csv_parameter.iat[0,4])+273.15 #the PV cell temperature under standard test conditions [25°C]

YPV = float(input_csv_parameter.iat[0,6]) #the rated capacity of the PV array, meaning its power output under standard test conditions [kW]
ap = float(input_csv_parameter.iat[0,7]) #Module Efficiency(Pmax)[%/℃]
beta= float(input_csv_parameter.iat[0,8])  #the slope of the surface [°]
eta_c = float(input_csv_parameter.iat[0,9]) #the solar absorptance and the solar transmittance[%]
fpv = float(input_csv_parameter.iat[0,10]) #the PV derating factor [%]
ganma = float(input_csv_parameter.iat[0,11]) #the azimuth of the surface [°]
lam=float(input_csv_parameter.iat[0,12])#the longitude [°]
phi=float(input_csv_parameter.iat[0,13])#the latitude [°]
rho_g = float(input_csv_parameter.iat[0,14]) # the ground reflectance, which is also called the albedo [%]
tau_a = float(input_csv_parameter.iat[0,15]) #the solar absorptance and the solar transmittance[%]
_Gt_STC =float(1) #the incident radiation at standard test conditions [1 kW/m2]
UL = (tau_a*Gt_NOCT)/(Tc_NOCT-Ta_NOCT)
#UL = float(input_csv_parameter.iat[0,5])
total_cost = float(input_csv_parameter.iat[0,16]) #total cost
Op_year = float(input_csv_parameter.iat[0,17]) #oparation year
cost = total_cost/(Op_year*365*24)#cost[yen/kW/h]

In [17]:
input_csv_parameter

,Gsc,Gt_NOCT,Ta_NOCT,Tc_NOCT,Tc_STC,UL,YPV,ap,beta,eta_c,fpv,ganma,lam,phi,rho_g,tau_a,total_cost,Op_year
0,1.367,0.8,20,44,25,3,194.44,-0.00258,33,0.194,0.8,0,130.25,33.349,0.2,0.9,199165,15


In [18]:
#ωの計算
forecast['ts'] = forecast['Hour']
forecast['ω'] = (forecast['ts'].values - 12 )*15#degree
forecast['n'] = Date['DateTime'] .dt.dayofyear

In [19]:
forecast['δ the solar declination']=23.45*np.sin(np.deg2rad(360*(284+forecast['n'].values)/365))#degree

delta=forecast['δ the solar declination'].values#degree

In [20]:
#角度をdeg2rad
omega = np.deg2rad(forecast['ω'].values)#rad
delta =np.deg2rad(delta)#rad
phi =math.radians(phi)#rad
beta =math.radians(beta)#rad
ganma =math.radians(ganma)#rad
lam =math.radians(lam)#rad

#cosθの計算
forecast['cosθ'] = np.sin(delta)*math.sin(phi)*math.cos(beta)\
                    -np.sin(delta)*math.cos(phi)*math.sin(beta)*math.cos(ganma)\
                    +np.cos(delta)*math.cos(phi)*math.cos(beta)*np.cos(omega)\
                    +np.cos(delta)*math.sin(phi)*math.sin(beta)*math.cos(ganma)*np.cos(omega)\
                    +np.cos(delta)*math.sin(beta)*math.sin(ganma)*np.sin(omega)
                    
cos_theta = forecast['cosθ'].values

In [21]:
#θの計算
forecast['θ(radian)'] = np.arccos(forecast['cosθ'].values)#rad

forecast['θ(degree)'] = np.rad2deg(forecast['θ(radian)'].values)#degree

Theta_deg = forecast['θ(radian)'].values#rad

#cosθzの計算
forecast['cosθz'] =math.cos(phi)*np.cos(delta)*np.cos(omega)\
                        +math.sin(phi)*np.sin(delta)

cos_theta_z = forecast['cosθz'].values

forecast['θz(radian)'] = np.arccos(forecast['cosθz'].values)#rad

forecast['θz(degree)'] = np.rad2deg(forecast['θz(radian)'].values)#degree

Theta_z_deg = forecast['θz(radian)'].values#rad

#theta degree ndarray
theta = forecast['θ(radian)'].values#rad

#theta degree ndarray
theta_z = forecast['θz(radian)'].values#rad

#decimals=1で小数点第2位を四捨五入する
cos_theta = np.round(cos_theta, decimals=1)
cos_theta_z = np.round(cos_theta_z, decimals=1)
Theta_deg =np.round(Theta_deg, decimals=0)
Theta_deg_z = np.round(Theta_z_deg, decimals=0)

cos_theta = np.cos(Theta_deg)
cos_theta_z = np.cos(Theta_z_deg)
forecast[:10]

,DateTime,Date,Year,Month,Day,Hour,wind_10m[m/s],index,temperature,index,...,ts,ω,n,δ the solar declination,cosθ,θ(radian),θ(degree),cosθz,θz(radian),θz(degree)
0,2019-04-21 21:00:00,2019-04-21 21:00:00,2019,4,21,21,0.536934,temperature FH0,15.915853,radiation FH0-1W/m2(avg),...,21,135,111,11.579037,-0.691481,2.334333,133.747443,-0.468309,2.058172,117.924578
1,2019-04-21 22:00:00,2019-04-21 22:00:00,2019,4,21,22,0.726766,temperature FH1,15.322592,radiation FH1-2W/m2(avg),...,22,150,111,11.579037,-0.847162,2.581418,147.904360,-0.598358,2.212246,126.752383
2,2019-04-21 23:00:00,2019-04-21 23:00:00,2019,4,21,23,1.124510,temperature FH2,15.305013,radiation FH2-3W/m2(avg),...,23,165,111,11.579037,-0.945028,2.808476,160.913829,-0.680110,2.318709,132.852253
3,2019-04-22 00:00:00,2019-04-22 00:00:00,2019,4,22,0,0.665433,temperature FH3,14.687673,radiation FH3-4W/m2(avg),...,0,-180,112,11.928334,-0.977130,2.927313,167.722666,-0.703676,2.351354,134.722666
4,2019-04-22 01:00:00,2019-04-22 01:00:00,2019,4,22,1,1.172903,temperature FH4,14.866750,radiation FH4-5W/m2(avg),...,1,-165,112,11.928334,-0.943792,2.804717,160.698446,-0.675827,2.312883,132.518406
5,2019-04-22 02:00:00,2019-04-22 02:00:00,2019,4,22,2,1.056662,temperature FH5,14.455984,radiation FH5-6W/m2(avg),...,2,-150,112,11.928334,-0.846051,2.579329,147.784667,-0.594178,2.207040,126.454085
6,2019-04-22 03:00:00,2019-04-22 03:00:00,2019,4,22,3,0.792906,temperature FH6,13.871572,radiation FH6-7W/m2(avg),...,3,-135,112,11.928334,-0.690566,2.333068,133.674957,-0.464294,2.053634,117.664554
7,2019-04-22 04:00:00,2019-04-22 04:00:00,2019,4,22,4,1.209161,temperature FH7,14.091635,radiation FH7-8W/m2(avg),...,4,-120,112,11.928334,-0.487935,2.080519,119.204972,-0.295026,1.870279,107.159087
8,2019-04-22 05:00:00,2019-04-22 05:00:00,2019,4,22,5,0.937732,temperature FH8,13.733907,radiation FH8-9W/m2(avg),...,5,-105,112,11.928334,-0.251967,1.825508,104.593920,-0.097909,1.668862,95.618754
9,2019-04-22 06:00:00,2019-04-22 06:00:00,2019,4,22,6,0.974783,temperature FH9,13.694204,radiation FH9-10W/m2(avg),...,6,-90,112,11.928334,0.001259,1.569537,89.927866,0.113624,1.456926,83.475725


In [22]:
#Gon(the extraterrestrial normal radiation)[kW/m2]の計算
forecast['Gon(the extraterrestrial normal radiation)[kW/m2]'] \
=Gsc*(1+0.033*np.cos(np.deg2rad(360*forecast['n'].values/365)))#np.deg2rad

Gon =forecast['Gon(the extraterrestrial normal radiation)[kW/m2]'].values

#Go(the extraterrestrial horizontal radiation [kW/m2])の計算
forecast['Go(the extraterrestrial horizontal radiation)[kW/m2]'] \
=Gon*np.abs(cos_theta_z)

Go = forecast['Go(the extraterrestrial horizontal radiation)[kW/m2]'] .values

In [23]:
#_Go(the extraterrestrial horizontal radiation averaged over the time step )[kW/m2]の計算

def step_minus(x):  #再帰関数　引数[2]-引数[1] ndarrayで返す ndarrayを引数
    x = list(x)
    y = [0]
    y.extend(x) #引数2
    x.extend([0]) #引数1
    y = np.array(y)
    x = np.array(x)
    ans = y - x
    return ans[1:]#[1:]or [:8760]


forecast['_Go(the extraterrestrial horizontal radiation averaged over the time step )[kW/m2]'] \
=(12/np.pi)*Gon*np.abs(np.cos(phi)*np.cos(delta)*step_minus(np.sin(omega))\
+((np.pi/180)*step_minus(omega)*np.sin(phi)*np.sin(delta)))

_Go = forecast['_Go(the extraterrestrial horizontal radiation averaged over the time step )[kW/m2]'] 

###start modified

In [24]:
forecast['the global horizontal radiation[kW/m2]']=\
forecast['radiation'].values/1000

_G = forecast['the global horizontal radiation[kW/m2]'].values
forecast['the global horizontal radiation[kW/m2]'] = _G 

In [25]:
#ktの計算
forecast['kt(the clearness index)'] =_G/_Go #_G2
kt = np.abs(forecast['kt(the clearness index)'].values)

_Go=kt/_G

#_Gdの計算
#２の３乗#累乗np.power(2, 3)
def if_kt(x):
    x = np.abs(x)
    ans = np.where(x>=0.8,0.165,np.where(x < 0.22, 1-0.09*x,0.95511-0.1604*x+4.388*x*x-16.638*x*x*x+12.336*x*x*x*x))
    return ans

forecast['_Gd(the diffuse radiation)[kW/m2]']=_G*if_kt(kt)
_Gd = forecast['_Gd(the diffuse radiation)[kW/m2]'].values

#_Gb(the beam radiation)の計算
forecast['_Gb(the beam radiation)[kW/m2]']=_G-_Gd

_Gb = forecast['_Gb(the beam radiation)[kW/m2]'].values

#_G
forecast['_G']=_Gd+_Gb

/home/yuichiro/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """
/home/yuichiro/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in greater_equal
  # This is added back by InteractiveShellApp.init_path()
/home/yuichiro/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in less
  # This is added back by InteractiveShellApp.init_path()


In [26]:
#Rbの計算
#input_data['Rb']= cos_theta/cos_theta_z
forecast['Rb']= np.abs(forecast['cosθ'].values/forecast['cosθz'].values)

#thetaの計算がjupyterだとうまくいかない泣
#input_data['Rb']=Rb
Rb = forecast['Rb'].values

#発散の処理 要考察 yを超えるとzになる　
def div(x,y,z):
    x = abs(x)
    ans = np.where(x>y,z,x)
    return ans
#Rb = div(Rb,3,3)
#input_data['Rb']= div(Rb,3,3)
forecast['Rb']= Rb

#Afの計算
forecast['Af']=_Gb/_Go #_G2

Af = forecast['Af'].values

#fの計算
forecast['f']=np.sqrt(_Gb/_G) #_G2
    #欠損値を0で置き換える
forecast=forecast.fillna(0)
f = forecast['f'].values                         

/home/yuichiro/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide


In [27]:
def sin3(x):
    return math.sin(x)*math.sin(x)*math.sin(x)

#_Gtの計算#２の３乗#累乗np.power(2, 3) np.power(np.sin(beta/2),3) or def 3sin(x)
_Gt=(_Gb+_Gd*Af)*Rb\
                +(_Gd*(1-Af)*((1+math.cos(beta))/2)\
                *np.abs(1+f*sin3(beta/2)))\
                +(_G*rho_g*((1-math.cos(beta))/2))
_Gt[np.isnan(_Gt)] = 0
        
forecast['_Gt']  = _Gt


In [28]:
#Tc(the PV cell temperature [°C])の計算
forecast['temperature[K]']=forecast['temperature']+ 273.15
Ta = forecast['temperature[K]'].values


#input_data['Tc(the PV cell temperature)[°C]']\
#= Ta+_Gt*(tau_a/UL)*(1-eta_c/tau_a)

forecast['Tc(the PV cell temperature)[K]']\
= Ta+_Gt*((Tc_NOCT-Ta_NOCT)/Gt_NOCT)*(1-eta_c/tau_a)

Tc = forecast['Tc(the PV cell temperature)[K]'].values

forecast['Tc(the PV cell temperature)[°C]']=forecast['Tc(the PV cell temperature)[K]']-273.15

#Ppv(the output of the PV array)[kW]の計算   #発散の処理div(x)を使用
#input_data['Ppv(the output of the PV array)[Wh]']\
#=div(YPV*fpv*(_Gt/_Gt_STC)*np.abs((1+ap*(Tc-Tc_STC))), YPV,YPV)

#Ppv(the output of the PV array)[kW]の計算   #発散の処理div(x)を使用2
forecast['Ppv(the output of the PV array)[Wh]']\
=YPV*fpv*(_Gt/_Gt_STC)*np.abs((1+ap*(Tc-Tc_STC)))

Ppv = forecast['Ppv(the output of the PV array)[Wh]'].values

#input_data['Left'] = YPV*fpv*(_Gt/_Gt_STC)
#input_data['Right'] =np.abs((1+ap*(Tc-Tc_STC)))

forecast[:10]

,DateTime,Date,Year,Month,Day,Hour,wind_10m[m/s],index,temperature,index,...,_Gb(the beam radiation)[kW/m2],_G,Rb,Af,f,_Gt,temperature[K],Tc(the PV cell temperature)[K],Tc(the PV cell temperature)[°C],Ppv(the output of the PV array)[Wh]
0,2019-04-21 21:00:00,2019-04-21 21:00:00,2019,4,21,21,0.536934,temperature FH0,15.915853,radiation FH0-1W/m2(avg),...,0.000000,0.000000,1.476549,0.000000e+00,0.000000,0.000000,289.065853,289.065853,15.915853,0.000000
1,2019-04-21 22:00:00,2019-04-21 22:00:00,2019,4,21,22,0.726766,temperature FH1,15.322592,radiation FH1-2W/m2(avg),...,0.000000,0.000000,1.415812,0.000000e+00,0.000000,0.000000,288.472592,288.472592,15.322592,0.000000
2,2019-04-21 23:00:00,2019-04-21 23:00:00,2019,4,21,23,1.124510,temperature FH2,15.305013,radiation FH2-3W/m2(avg),...,0.000000,0.000000,1.389522,0.000000e+00,0.000000,0.000000,288.455013,288.455013,15.305013,0.000000
3,2019-04-22 00:00:00,2019-04-22 00:00:00,2019,4,22,0,0.665433,temperature FH3,14.687673,radiation FH3-4W/m2(avg),...,0.000000,0.000000,1.388608,0.000000e+00,0.000000,0.000000,287.837673,287.837673,14.687673,0.000000
4,2019-04-22 01:00:00,2019-04-22 01:00:00,2019,4,22,1,1.172903,temperature FH4,14.866750,radiation FH4-5W/m2(avg),...,0.000000,0.000000,1.396499,0.000000e+00,0.000000,0.000000,288.016750,288.016750,14.866750,0.000000
5,2019-04-22 02:00:00,2019-04-22 02:00:00,2019,4,22,2,1.056662,temperature FH5,14.455984,radiation FH5-6W/m2(avg),...,0.000000,0.000000,1.423900,0.000000e+00,0.000000,0.000000,287.605984,287.605984,14.455984,0.000000
6,2019-04-22 03:00:00,2019-04-22 03:00:00,2019,4,22,3,0.792906,temperature FH6,13.871572,radiation FH6-7W/m2(avg),...,0.000000,0.000000,1.487346,0.000000e+00,0.000000,0.000000,287.021572,287.021572,13.871572,0.000000
7,2019-04-22 04:00:00,2019-04-22 04:00:00,2019,4,22,4,1.209161,temperature FH7,14.091635,radiation FH7-8W/m2(avg),...,0.000000,0.000000,1.653873,0.000000e+00,0.000000,0.000000,287.241635,287.241635,14.091635,0.000000
8,2019-04-22 05:00:00,2019-04-22 05:00:00,2019,4,22,5,0.937732,temperature FH8,13.733907,radiation FH8-9W/m2(avg),...,0.000001,0.001437,2.573487,1.859766e-07,0.030285,0.001348,286.883907,286.915626,13.765626,0.215735
9,2019-04-22 06:00:00,2019-04-22 06:00:00,2019,4,22,6,0.974783,temperature FH9,13.694204,radiation FH9-10W/m2(avg),...,0.007123,0.055644,0.011080,1.042879e-03,0.357778,0.045903,286.844204,287.924458,14.774458,7.328698


In [29]:
#入力初期値をデータフレームにする。
input_parameter_name=['Gsc','Gt_NOCT','Ta_NOCT','Tc_NOCT','Tc_STC','UL',\
                      'YPV','ap','beta','eta_c','fpv','ganma','lam','phi',\
                      'rho_g','tau_a','total_cost','Op_year','cost']

input_parameter_list=[Gsc,Gt_NOCT,Ta_NOCT,Tc_NOCT,Tc_STC,UL,\
                      YPV,ap,beta,eta_c,fpv,ganma,lam,phi,\
                      rho_g,tau_a,total_cost,Op_year,cost]

def float2DataFrame(name,list):
    df0=pd.DataFrame({str(name[0]): [list[0]]})
    for i in range(len(list)-1):
        a = float(list[i+1])
        df=pd.DataFrame({str(name[i+1]): [a]})
        df0 =pd.concat([df0, df], axis=1)
    return df0

input_parameter=float2DataFrame(input_parameter_name,input_parameter_list)
input_parameter

,Gsc,Gt_NOCT,Ta_NOCT,Tc_NOCT,Tc_STC,UL,YPV,ap,beta,eta_c,fpv,ganma,lam,phi,rho_g,tau_a,total_cost,Op_year,cost
0,1.367,0.8,293.15,317.15,298.15,0.03,194.44,-0.00258,0.575959,0.194,0.8,0.0,2.273291,0.58205,0.2,0.9,199165.0,15.0,1.515715


In [30]:
#Timeをインデックスに設定ものをdfと定義
#df = input_data.set_index("Time") 
input_parameter.to_csv('PV_input_parameter_result.csv',encoding="SHIFT-JIS")  

In [31]:
df = forecast.set_index('DateTime')
df.columns

Index(['Date', 'Year', 'Month', 'Day', 'Hour', 'wind_10m[m/s]', 'index',
       'temperature', 'index', 'radiation', 'index', 'Uwind', 'index', 'Vwind',
       'wind_30m[m/s]', 'wind output[kWh]', 'ts', 'ω', 'n',
       'δ the solar declination', 'cosθ', 'θ(radian)', 'θ(degree)', 'cosθz',
       'θz(radian)', 'θz(degree)',
       'Gon(the extraterrestrial normal radiation)[kW/m2]',
       'Go(the extraterrestrial horizontal radiation)[kW/m2]',
       '_Go(the extraterrestrial horizontal radiation averaged over the time step )[kW/m2]',
       'the global horizontal radiation[kW/m2]', 'kt(the clearness index)',
       '_Gd(the diffuse radiation)[kW/m2]', '_Gb(the beam radiation)[kW/m2]',
       '_G', 'Rb', 'Af', 'f', '_Gt', 'temperature[K]',
       'Tc(the PV cell temperature)[K]', 'Tc(the PV cell temperature)[°C]',
       'Ppv(the output of the PV array)[Wh]'],
      dtype='object')

In [32]:
forecast['Rb'].describe()

count    52.000000
mean      1.234488
std       0.575784
min       0.011080
25%       0.997387
50%       1.220473
75%       1.443151
max       2.665533
Name: Rb, dtype: float64

# Visualizing

In [33]:
#データの書き出し
filename = file_dir_path+'/PV_result_forecast'+year+month+day+time+'.csv'
forecast.to_csv(filename,encoding="SHIFT-JIS")  
